In [2]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from surrogate import ImageSurrogate

In [3]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds

from datetime import datetime
import os

In [5]:
from tensorflow.keras.layers import (Input, Layer, Dense)
from tensorflow.keras.models import Model

In [6]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [7]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Load Data

In [8]:
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'imagenette/full-size-v2',
    split=['train', 'validation[:50%]', 'validation[-50%:]'],
    as_supervised=False,
    with_info=True
)

### Batch Data

In [9]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [10]:
BATCH_SIZE = 32

def reformat(input_dict):
    
    i = input_dict['image']
    i = tf.cast(i, tf.float32)
    i = tf.image.resize_with_crop_or_pad(i, 224, 224)
    i = tf.keras.applications.resnet50.preprocess_input(i)
    
    l = tf.one_hot(input_dict['label'], depth = 10)
    
    return (i, l)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

## Load Model

In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50

INPUT_SHAPE = (224,224,3)

base_model = ResNet50(
    include_top=True, weights='imagenet', 
    input_shape=INPUT_SHAPE
)
base_model.trainable = False

model_input = Input(shape=INPUT_SHAPE, dtype='float32', name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = 'model/20210511_21_28_36/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

# Train Surrogate

### Save Dir

In [12]:
date = datetime.now().strftime("%Y%m%d_%H_%M_%S")
save_dir = 'surrogate'
model_dir = os.path.join(os.getcwd(), save_dir, date)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Train

In [33]:
surrogate = ImageSurrogate(model, model_dir)

t = time.time()
surrogate.train(train_data = ds_train, 
                val_data = ds_val, 
                max_epochs = 100, 
                batch_size = 32, 
                lookback = 10,
                lr = 1e-2)
training_time = time.time() - t

with open(os.path.join(model_dir, 'training_time.pkl'), 'wb') as f:
    pickle.dump(training_time, f)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 1/100
296/296 [==============================] - ETA: 0s - loss: 2.7022 - accuracy: 0.1378
Epoch 00001: val_loss improved from inf to 2.30642, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/imagenette/surrogate/20210511_21_47_45/value_weights.h5
296/296 [==============================] - 117s 395ms/step - loss: 2.7022 - accuracy: 0.1378 - val_loss: 2.3064 - val_accuracy: 0.1473
Epoch 2/100
296/296 [==============================] - ETA: 0s - loss: 2.1637 - accuracy: 0.2050
Epoch 00002: val_loss did not improve from 2.30642
296/296 [==============================] - 46s 155ms/step - loss: 2.1637 - accuracy: 0.2050 - val_loss: 3.2643 - val_accuracy: 0.1621
Epoch 3/100
296/296 [==============================] - ETA: 0s - loss: 2.0054 - accuracy: 0.3027
Epoch 00003: val_loss did not improve from 2.30642
296/296 [==============================] - 46s 155ms/step - loss: 2.0054 - accuracy: 0.3027 - val_loss: 2.6346 - val_accuracy: 0.2574
Epoch 4/100
296/296 [======

Epoch 26/100
296/296 [==============================] - ETA: 0s - loss: 0.7881 - accuracy: 0.7560
Epoch 00026: val_loss did not improve from 1.49540
296/296 [==============================] - 46s 156ms/step - loss: 0.7881 - accuracy: 0.7560 - val_loss: 1.5659 - val_accuracy: 0.5489
Epoch 27/100
296/296 [==============================] - ETA: 0s - loss: 0.7791 - accuracy: 0.7612
Epoch 00027: val_loss improved from 1.49540 to 1.44669, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/imagenette/surrogate/20210511_21_47_45/value_weights.h5
296/296 [==============================] - 46s 156ms/step - loss: 0.7791 - accuracy: 0.7612 - val_loss: 1.4467 - val_accuracy: 0.5724
Epoch 28/100
296/296 [==============================] - ETA: 0s - loss: 0.7538 - accuracy: 0.7717
Epoch 00028: val_loss improved from 1.44669 to 1.44219, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/imagenette/surrogate/20210511_21_47_45/value_weights.h5
296/296 [================

Epoch 51/100
296/296 [==============================] - ETA: 0s - loss: 0.5953 - accuracy: 0.8261
Epoch 00051: val_loss did not improve from 1.26103
296/296 [==============================] - 44s 150ms/step - loss: 0.5953 - accuracy: 0.8261 - val_loss: 1.4805 - val_accuracy: 0.5841
Epoch 52/100
296/296 [==============================] - ETA: 0s - loss: 0.6038 - accuracy: 0.8217
Epoch 00052: val_loss did not improve from 1.26103
296/296 [==============================] - 45s 150ms/step - loss: 0.6038 - accuracy: 0.8217 - val_loss: 1.2725 - val_accuracy: 0.6279
Epoch 53/100
296/296 [==============================] - ETA: 0s - loss: 0.5843 - accuracy: 0.8263
Epoch 00053: val_loss did not improve from 1.26103

Epoch 00053: ReduceLROnPlateau reducing learning rate to 0.003138105757534504.
296/296 [==============================] - 45s 150ms/step - loss: 0.5843 - accuracy: 0.8263 - val_loss: 1.3039 - val_accuracy: 0.6162
Epoch 54/100
296/296 [==============================] - ETA: 0s - loss:

Epoch 76/100
296/296 [==============================] - ETA: 0s - loss: 0.5379 - accuracy: 0.8424
Epoch 00076: val_loss did not improve from 1.12409
296/296 [==============================] - 44s 150ms/step - loss: 0.5379 - accuracy: 0.8424 - val_loss: 1.1394 - val_accuracy: 0.6621
Epoch 77/100
296/296 [==============================] - ETA: 0s - loss: 0.5271 - accuracy: 0.8489
Epoch 00077: val_loss did not improve from 1.12409
296/296 [==============================] - 44s 150ms/step - loss: 0.5271 - accuracy: 0.8489 - val_loss: 1.1552 - val_accuracy: 0.6468
Epoch 78/100
296/296 [==============================] - ETA: 0s - loss: 0.5068 - accuracy: 0.8548
Epoch 00078: val_loss did not improve from 1.12409

Epoch 00078: ReduceLROnPlateau reducing learning rate to 0.0018530200235545636.
296/296 [==============================] - 44s 150ms/step - loss: 0.5068 - accuracy: 0.8548 - val_loss: 1.2442 - val_accuracy: 0.6509
Epoch 79/100
296/296 [==============================] - ETA: 0s - loss